In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pathlib
import plotly
import plotly_express as px
import plotly.graph_objs as go
import iplot
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import umap
from sklearn import preprocessing

In [2]:
nELISA_data_path = pathlib.Path("./nELISA_Data_UCA1_2023.04.11.csv")
MetaData_path = pathlib.Path("./Metadata_UCA1_2023.04.11.csv")

nELISA_data_all = pd.read_csv(nELISA_data_path)
MetaData = pd.read_csv(MetaData_path)

In [3]:
# Change the 'A1' cell format to 'A01' format
position = []
for i in MetaData["position"].astype(str):
    position.append(i[:1] + f"{0}" + i[1:])
MetaData["position"] = position
MetaData.head()

,plate_name,plate_barcode,position,cell_type,incubation inducer,inhibitor,inhibitor_concentration_value,inhibitor_concentration_unit,inhibitor_concentration,inducer1,inducer1_concentration_value,inducer1_concentration_unit,inducer1_concentration,inducer2,inducer2_concentration_value,inducer2_concentration_unit,inducer2_concentration
0,70117_20230210MM1_P1,430420,B06,PBMC,6 h,DMSO,0.025,%,0.03%,DMSO,0.1,%,0.10%,NaN,NaN,NaN,NaN
1,70117_20230210MM1_P1,430420,C06,PBMC,6 h,DMSO,0.025,%,0.03%,DMSO,0.1,%,0.10%,NaN,NaN,NaN,NaN
2,70117_20230210MM1_P1,430420,I06,PBMC,6 h,DMSO,0.025,%,0.03%,DMSO,0.1,%,0.10%,NaN,NaN,NaN,NaN
3,70117_20230210MM1_P1,430420,J06,PBMC,6 h,DMSO,0.025,%,0.03%,DMSO,0.1,%,0.10%,NaN,NaN,NaN,NaN
4,70117_20230210MM1_P1,430420,B07,PBMC,6 h,DMSO,0.025,%,0.03%,DMSO,0.1,%,0.10%,NaN,NaN,NaN,NaN


In [4]:
# nELISA_data = nELISA_data_all.filter(like="NSU", axis=1)

# nELISA_data = preprocessing.normalize(nELISA_data)

# display max columns
pd.set_option("display.max_columns", None)
# display max rows
pd.set_option("display.max_rows", None)
nELISA_data_all = nELISA_data_all.rename({"user_well_loc": "position"}, axis=1)
nELISA_data_all.head()
# MetaData.head()
# nELISA_data = pd.concat([MetaData,nELISA_data_all], axis=1)
# nELISA_data

MetaData["plate_position"] = (
    MetaData["plate_barcode"].astype(str) + "_" + MetaData["position"].astype(str)
)
MetaData["plate_position"]

nELISA_data_all["plate_position"] = (
    nELISA_data_all["user_plate_id"].astype(str)
    + "_"
    + nELISA_data_all["position"].astype(str)
)
print(nELISA_data_all["user_plate_id"].unique())
nELISA_data_all.head()
print(MetaData["plate_barcode"].unique())

nELISA_data_all.replace(regex=[" and "], value="_", inplace=True)
print(nELISA_data_all["user_plate_id"].unique())

['430420' '430418 and 430419']
['430420' '430418_430419']
['430420' '430418_430419']


In [5]:
# nELISA_data[nELISA_data['nelisa_sample_comments'].str.contains('empty_well')==False]

nELISA_data_all = nELISA_data_all[
    ~nELISA_data_all.nelisa_sample_comments.str.contains("empty_well", na=False)
]
nELISA_data_all = nELISA_data_all[
    ~nELISA_data_all.nelisa_sample_comments.str.contains("qc_fail", na=False)
]

# nELISA_data_all = nELISA_data_all.filter(like="NSU", axis=1)
# MetaData = MetaData.astype("float")
nELISA_data_all.head()

,user_plate_id,user_sample_id,position,nelisa_sample_dilution_fold,nelisa_sample_comments,nelisa_sample_pass_qc,nelisa_sample_id,Activin A [NSU],AITRL (GITR Ligand) [NSU],Amphiregulin [NSU],Amyloid beta [NSU],APRIL [NSU],BAFF [NSU],BCMA (TNFRSF17) [NSU],BDNF [NSU],BMP2 [NSU],BMP3 [NSU],BMP4 [NSU],BMP6 [NSU],BMP7 [NSU],BMP9 [NSU],C5/C5a [NSU],Calbindin [NSU],CCL1 [NSU],CCL11 [NSU],CCL13 [NSU],CCL15 [NSU],CCL16 [NSU],CCL17 [NSU],CCL18 [NSU],CCL19 [NSU],CCL2 [NSU],CCL20 [NSU],CCL21 [NSU],CCL22 [NSU],CCL23 [NSU],CCL24 [NSU],CCL25 [NSU],CCL27 [NSU],CCL28 [NSU],CCL3 [NSU],CCL4 [NSU],CCL5 [NSU],CCL7 [NSU],CCL8 [NSU],CD14 [NSU],CD163 [NSU],CD276 (B7-H3) [NSU],CD27L [NSU],CD30 [NSU],CD40L [NSU],CNTF [NSU],CRP [NSU],CX3CL1 [NSU],CXCL1 [NSU],CXCL10 [NSU],CXCL11 [NSU],CXCL12 (alpha) [NSU],CXCL12 (beta) [NSU],CXCL13 [NSU],CXCL14 [NSU],CXCL16 [NSU],CXCL17 [NSU],CXCL3 [NSU],CXCL4 [NSU],CXCL5 [NSU],CXCL6 [NSU],CXCL7 [NSU],CXCL9 [NSU],Cytochrome C [NSU],EGF [NSU],EGFR [NSU],EMMPRIN [NSU],FAS-L [NSU],FGF-1 [NSU],FGF-19 [NSU],FGF-2 [NSU],FGF-21 [NSU],FGF-4 [NSU],FGF-6 [NSU],FGF-7 (KGF) [NSU],FGF-9 [NSU],FGFR3 (IIIc) [NSU],FLRG (FSTL3) [NSU],Flt-3 Ligand [NSU],G-CSF [NSU],GDF-11 (BMP-11) [NSU],GDF-15 (MIC-1) [NSU],GDNF [NSU],GM-CSF [NSU],Granzyme B [NSU],Growth Hormone (Somatotropin) [NSU],HGF [NSU],HVEM [NSU],ICAM-1 [NSU],ICAM-2 [NSU],IFN alpha 2 (alpha 2b) [NSU],IFN beta [NSU],IFN gamma [NSU],IFN-epsilon [NSU],IGF-1 [NSU],IL-1 alpha [NSU],IL-1 beta [NSU],IL-1 R1 [NSU],IL-1 RA/RN [NSU],IL-10 [NSU],IL-11 [NSU],IL-12 p35 [NSU],IL-12 p40 [NSU],IL-12 p70 [NSU],IL-15 [NSU],IL-15/IL-15R alpha complex [NSU],IL-16 [NSU],IL-17A [NSU],IL-17B [NSU],IL-17C [NSU],IL-17D [NSU],IL-17E (IL-25) [NSU],IL-17F [NSU],IL-18 [NSU],IL-2 [NSU],IL-2 RA [NSU],IL-21 [NSU],IL-22 [NSU],IL-22 BP [NSU],IL-23 [NSU],IL-24 [NSU],IL-27 [NSU],IL-28A [NSU],IL-29 [NSU],IL-3 [NSU],IL-31 [NSU],IL-32 (alpha) [NSU],IL-33 [NSU],IL-35 [NSU],IL-4 [NSU],IL-5 [NSU],IL-6 [NSU],IL-6 R alpha [NSU],IL-7 [NSU],IL-8 [NSU],IL-9 [NSU],Leptin [NSU],LIF [NSU],LOX1 (OLR1) [NSU],M-CSF [NSU],M-CSF R (CD115) [NSU],Mesothelin [NSU],MIF [NSU],MMP-1 [NSU],MMP-10 [NSU],MMP-12 [NSU],MMP-2 [NSU],MMP-3 [NSU],MMP-7 [NSU],MMP-9 [NSU],NF-L [NSU],NGF beta [NSU],NRG1 beta 1 [NSU],Oncostatin M (OSM) [NSU],Osteopontin (OPN) [NSU],PCSK9 [NSU],PDGF-BB [NSU],PLGF [NSU],PTX3 (Pentraxin 3) [NSU],Resistin [NSU],SAA [NSU],SCF [NSU],ST2 (IL-33R) [NSU],TGF-beta 1 (LAP domain in precursor) [NSU],TGF-beta 1 (total) [NSU],TGF-beta 2 [NSU],TGF-beta 3 [NSU],Tie-2 [NSU],TIMP1 [NSU],Tissue Factor (TF) [NSU],TNF alpha [NSU],TNF RI [NSU],TNF RII [NSU],TNF RIII (Lymphotoxin Beta R) [NSU],TPO (Thrombopoietin) [NSU],TRAIL [NSU],TREM2 [NSU],TSLP [NSU],TWEAK [NSU],uPA [NSU],VCAM-1 [NSU],VEGF Receptor 2 (Flk-1) [NSU],VEGF-A (165) [NSU],VEGF-C [NSU],VEGF-D [NSU],VEGFR-1 [NSU],WISP-1 (CCN4) [NSU],XCL1 (Lymphotactin) [NSU],Activin A [pgML],AITRL (GITR Ligand) [pgML],Amphiregulin [pgML],Amyloid beta [pgML],APRIL [pgML],BAFF [pgML],BCMA (TNFRSF17) [pgML],BDNF [pgML],BMP2 [pgML],BMP3 [pgML],BMP4 [pgML],BMP6 [pgML],BMP7 [pgML],BMP9 [pgML],C5/C5a [pgML],Calbindin [pgML],CCL1 [pgML],CCL11 [pgML],CCL13 [pgML],CCL15 [pgML],CCL16 [pgML],CCL17 [pgML],CCL18 [pgML],CCL19 [pgML],CCL2 [pgML],CCL20 [pgML],CCL21 [pgML],CCL22 [pgML],CCL23 [pgML],CCL24 [pgML],CCL25 [pgML],CCL27 [pgML],CCL28 [pgML],CCL3 [pgML],CCL4 [pgML],CCL5 [pgML],CCL7 [pgML],CCL8 [pgML],CD14 [pgML],CD163 [pgML],CD276 (B7-H3) [pgML],CD27L [pgML],CD30 [pgML],CD40L [pgML],CNTF [pgML],CRP [pgML],CX3CL1 [pgML],CXCL1 [pgML],CXCL10 [pgML],CXCL11 [pgML],CXCL12 (alpha) [pgML],CXCL12 (beta) [pgML],CXCL13 [pgML],CXCL14 [pgML],CXCL16 [pgML],CXCL17 [pgML],CXCL3 [pgML],CXCL4 [pgML],CXCL5 [pgML],CXCL6 [pgML],CXCL7 [pgML],CXCL9 [pgML],Cytochrome C [pgML],EGF [pgML],EGFR [pgML],EMMPRIN [pgML],FAS-L [pgML],FGF-1 [pgML],FGF-19 [pgML],FGF-2 [pgML],FGF-21 [pgML],FGF-4 [pgML],FGF-6 [pgML],FGF-7 (KGF) [pgML],FGF-9 [pgML],FGFR3 (IIIc) [pgML],FLRG (FSTL3) [pgML],Flt-3 Ligand [pgML],G-CSF [pgML],GDF-11 (BMP-11) [pgML],G

In [6]:
MetaData.head()

,plate_name,plate_barcode,position,cell_type,incubation inducer,inhibitor,inhibitor_concentration_value,inhibitor_concentration_unit,inhibitor_concentration,inducer1,inducer1_concentration_value,inducer1_concentration_unit,inducer1_concentration,inducer2,inducer2_concentration_value,inducer2_concentration_unit,inducer2_concentration,plate_position
0,70117_20230210MM1_P1,430420,B06,PBMC,6 h,DMSO,0.025,%,0.03%,DMSO,0.1,%,0.10%,NaN,NaN,NaN,NaN,430420_B06
1,70117_20230210MM1_P1,430420,C06,PBMC,6 h,DMSO,0.025,%,0.03%,DMSO,0.1,%,0.10%,NaN,NaN,NaN,NaN,430420_C06
2,70117_20230210MM1_P1,430420,I06,PBMC,6 h,DMSO,0.025,%,0.03%,DMSO,0.1,%,0.10%,NaN,NaN,NaN,NaN,430420_I06
3,70117_20230210MM1_P1,430420,J06,PBMC,6 h,DMSO,0.025,%,0.03%,DMSO,0.1,%,0.10%,NaN,NaN,NaN,NaN,430420_J06
4,70117_20230210MM1_P1,430420,B07,PBMC,6 h,DMSO,0.025,%,0.03%,DMSO,0.1,%,0.10%,NaN,NaN,NaN,NaN,430420_B07


In [7]:
nELISA_data_all["plate_position"].unique().shape
MetaData["plate_position"].unique().shape

(768,)

In [8]:
# nELISA_data = pd.merge(MetaData, nELISA_data_all, left_on='plate_position',right_on='plate_position', how='inner')
nELISA_data = pd.merge(MetaData, nELISA_data_all, on="plate_position", how="right")
nELISA_data_values = nELISA_data.filter(like="NSU", axis=1)
nELISA_data_values = nELISA_data_values.astype("float")
print(nELISA_data_values.shape)
# pd.merge(MetaData, nELISA_data_all, on='plate_position',how='left').shape
nELISA_data_values.head()

(689, 187)


,Activin A [NSU],AITRL (GITR Ligand) [NSU],Amphiregulin [NSU],Amyloid beta [NSU],APRIL [NSU],BAFF [NSU],BCMA (TNFRSF17) [NSU],BDNF [NSU],BMP2 [NSU],BMP3 [NSU],BMP4 [NSU],BMP6 [NSU],BMP7 [NSU],BMP9 [NSU],C5/C5a [NSU],Calbindin [NSU],CCL1 [NSU],CCL11 [NSU],CCL13 [NSU],CCL15 [NSU],CCL16 [NSU],CCL17 [NSU],CCL18 [NSU],CCL19 [NSU],CCL2 [NSU],CCL20 [NSU],CCL21 [NSU],CCL22 [NSU],CCL23 [NSU],CCL24 [NSU],CCL25 [NSU],CCL27 [NSU],CCL28 [NSU],CCL3 [NSU],CCL4 [NSU],CCL5 [NSU],CCL7 [NSU],CCL8 [NSU],CD14 [NSU],CD163 [NSU],CD276 (B7-H3) [NSU],CD27L [NSU],CD30 [NSU],CD40L [NSU],CNTF [NSU],CRP [NSU],CX3CL1 [NSU],CXCL1 [NSU],CXCL10 [NSU],CXCL11 [NSU],CXCL12 (alpha) [NSU],CXCL12 (beta) [NSU],CXCL13 [NSU],CXCL14 [NSU],CXCL16 [NSU],CXCL17 [NSU],CXCL3 [NSU],CXCL4 [NSU],CXCL5 [NSU],CXCL6 [NSU],CXCL7 [NSU],CXCL9 [NSU],Cytochrome C [NSU],EGF [NSU],EGFR [NSU],EMMPRIN [NSU],FAS-L [NSU],FGF-1 [NSU],FGF-19 [NSU],FGF-2 [NSU],FGF-21 [NSU],FGF-4 [NSU],FGF-6 [NSU],FGF-7 (KGF) [NSU],FGF-9 [NSU],FGFR3 (IIIc) [NSU],FLRG (FSTL3) [NSU],Flt-3 Ligand [NSU],G-CSF [NSU],GDF-11 (BMP-11) [NSU],GDF-15 (MIC-1) [NSU],GDNF [NSU],GM-CSF [NSU],Granzyme B [NSU],Growth Hormone (Somatotropin) [NSU],HGF [NSU],HVEM [NSU],ICAM-1 [NSU],ICAM-2 [NSU],IFN alpha 2 (alpha 2b) [NSU],IFN beta [NSU],IFN gamma [NSU],IFN-epsilon [NSU],IGF-1 [NSU],IL-1 alpha [NSU],IL-1 beta [NSU],IL-1 R1 [NSU],IL-1 RA/RN [NSU],IL-10 [NSU],IL-11 [NSU],IL-12 p35 [NSU],IL-12 p40 [NSU],IL-12 p70 [NSU],IL-15 [NSU],IL-15/IL-15R alpha complex [NSU],IL-16 [NSU],IL-17A [NSU],IL-17B [NSU],IL-17C [NSU],IL-17D [NSU],IL-17E (IL-25) [NSU],IL-17F [NSU],IL-18 [NSU],IL-2 [NSU],IL-2 RA [NSU],IL-21 [NSU],IL-22 [NSU],IL-22 BP [NSU],IL-23 [NSU],IL-24 [NSU],IL-27 [NSU],IL-28A [NSU],IL-29 [NSU],IL-3 [NSU],IL-31 [NSU],IL-32 (alpha) [NSU],IL-33 [NSU],IL-35 [NSU],IL-4 [NSU],IL-5 [NSU],IL-6 [NSU],IL-6 R alpha [NSU],IL-7 [NSU],IL-8 [NSU],IL-9 [NSU],Leptin [NSU],LIF [NSU],LOX1 (OLR1) [NSU],M-CSF [NSU],M-CSF R (CD115) [NSU],Mesothelin [NSU],MIF [NSU],MMP-1 [NSU],MMP-10 [NSU],MMP-12 [NSU],MMP-2 [NSU],MMP-3 [NSU],MMP-7 [NSU],MMP-9 [NSU],NF-L [NSU],NGF beta [NSU],NRG1 beta 1 [NSU],Oncostatin M (OSM) [NSU],Osteopontin (OPN) [NSU],PCSK9 [NSU],PDGF-BB [NSU],PLGF [NSU],PTX3 (Pentraxin 3) [NSU],Resistin [NSU],SAA [NSU],SCF [NSU],ST2 (IL-33R) [NSU],TGF-beta 1 (LAP domain in precursor) [NSU],TGF-beta 1 (total) [NSU],TGF-beta 2 [NSU],TGF-beta 3 [NSU],Tie-2 [NSU],TIMP1 [NSU],Tissue Factor (TF) [NSU],TNF alpha [NSU],TNF RI [NSU],TNF RII [NSU],TNF RIII (Lymphotoxin Beta R) [NSU],TPO (Thrombopoietin) [NSU],TRAIL [NSU],TREM2 [NSU],TSLP [NSU],TWEAK [NSU],uPA [NSU],VCAM-1 [NSU],VEGF Receptor 2 (Flk-1) [NSU],VEGF-A (165) [NSU],VEGF-C [NSU],VEGF-D [NSU],VEGFR-1 [NSU],WISP-1 (CCN4) [NSU],XCL1 (Lymphotactin) [NSU]
0,26.491764,-1.073383,25.817862,-2.435479,-2.556473,-1.259130,-0.043782,-0.661045,-1.508780,-2.046371,0.168079,-1.706849,-0.777835,4.007890,0.352881,-2.222222,-0.237105,-1.542583,-0.457152,-0.492541,2.718748,2.836587,0.649378,1.962963,162.157574,53.071640,0.710012,10.961222,0.432067,5.804066,-1.477359,0.258294,-3.709112,206.909867,183.686021,73.439111,45.735417,18.211329,19.462327,0.736303,-3.124059,0.620207,-0.154579,-1.555782,-0.345606,0.328923,1.506642,157.596660,112.112899,31.153619,-0.989890,-0.042750,2.370725,-0.863558,95.861127,-1.157654,41.218921,89.237405,130.530930,-0.666730,50.567654,31.628723,-0.270998,0.364409,-1.262286,10.190143,-0.117047,-1.012891,-0.745658,-1.032250,-1.999676,-1.059526,0.889647,0.029996,-1.448270,-0.670735,3.375657,2.938226,33.393544,-0.800489,-1.458865,-0.381225,2.191499,-1.933073,0.986341,-0.849175,0.832019,1.589426,-0.263385,0.338542,3.160363,111.892305,-1.492156,-2.155013,53.764101,103.187771,0.338297,29.367143,8.053770,-1.562862,-0.092570,33.177833,-0.279728,1.775209,0.368577,8.507318,-1.171827,0.227443,-0.608019,-2.147776,-1.278807,-1.220966,4.492879,-0.178313,-0.578333,-1.739603,1.250734,0.220366,-1.416586,0.408523,0.356067,0.979887,-0.446829,-2.192593,1.061541,-2.907188,-0.333802,-2.230838,-1.961364,0.694773,130.

In [9]:
max_values = nELISA_data_values.max()
nELISA_data_values = nELISA_data_values.div(max_values)
nELISA_data_values.head(10)

,Activin A [NSU],AITRL (GITR Ligand) [NSU],Amphiregulin [NSU],Amyloid beta [NSU],APRIL [NSU],BAFF [NSU],BCMA (TNFRSF17) [NSU],BDNF [NSU],BMP2 [NSU],BMP3 [NSU],BMP4 [NSU],BMP6 [NSU],BMP7 [NSU],BMP9 [NSU],C5/C5a [NSU],Calbindin [NSU],CCL1 [NSU],CCL11 [NSU],CCL13 [NSU],CCL15 [NSU],CCL16 [NSU],CCL17 [NSU],CCL18 [NSU],CCL19 [NSU],CCL2 [NSU],CCL20 [NSU],CCL21 [NSU],CCL22 [NSU],CCL23 [NSU],CCL24 [NSU],CCL25 [NSU],CCL27 [NSU],CCL28 [NSU],CCL3 [NSU],CCL4 [NSU],CCL5 [NSU],CCL7 [NSU],CCL8 [NSU],CD14 [NSU],CD163 [NSU],CD276 (B7-H3) [NSU],CD27L [NSU],CD30 [NSU],CD40L [NSU],CNTF [NSU],CRP [NSU],CX3CL1 [NSU],CXCL1 [NSU],CXCL10 [NSU],CXCL11 [NSU],CXCL12 (alpha) [NSU],CXCL12 (beta) [NSU],CXCL13 [NSU],CXCL14 [NSU],CXCL16 [NSU],CXCL17 [NSU],CXCL3 [NSU],CXCL4 [NSU],CXCL5 [NSU],CXCL6 [NSU],CXCL7 [NSU],CXCL9 [NSU],Cytochrome C [NSU],EGF [NSU],EGFR [NSU],EMMPRIN [NSU],FAS-L [NSU],FGF-1 [NSU],FGF-19 [NSU],FGF-2 [NSU],FGF-21 [NSU],FGF-4 [NSU],FGF-6 [NSU],FGF-7 (KGF) [NSU],FGF-9 [NSU],FGFR3 (IIIc) [NSU],FLRG (FSTL3) [NSU],Flt-3 Ligand [NSU],G-CSF [NSU],GDF-11 (BMP-11) [NSU],GDF-15 (MIC-1) [NSU],GDNF [NSU],GM-CSF [NSU],Granzyme B [NSU],Growth Hormone (Somatotropin) [NSU],HGF [NSU],HVEM [NSU],ICAM-1 [NSU],ICAM-2 [NSU],IFN alpha 2 (alpha 2b) [NSU],IFN beta [NSU],IFN gamma [NSU],IFN-epsilon [NSU],IGF-1 [NSU],IL-1 alpha [NSU],IL-1 beta [NSU],IL-1 R1 [NSU],IL-1 RA/RN [NSU],IL-10 [NSU],IL-11 [NSU],IL-12 p35 [NSU],IL-12 p40 [NSU],IL-12 p70 [NSU],IL-15 [NSU],IL-15/IL-15R alpha complex [NSU],IL-16 [NSU],IL-17A [NSU],IL-17B [NSU],IL-17C [NSU],IL-17D [NSU],IL-17E (IL-25) [NSU],IL-17F [NSU],IL-18 [NSU],IL-2 [NSU],IL-2 RA [NSU],IL-21 [NSU],IL-22 [NSU],IL-22 BP [NSU],IL-23 [NSU],IL-24 [NSU],IL-27 [NSU],IL-28A [NSU],IL-29 [NSU],IL-3 [NSU],IL-31 [NSU],IL-32 (alpha) [NSU],IL-33 [NSU],IL-35 [NSU],IL-4 [NSU],IL-5 [NSU],IL-6 [NSU],IL-6 R alpha [NSU],IL-7 [NSU],IL-8 [NSU],IL-9 [NSU],Leptin [NSU],LIF [NSU],LOX1 (OLR1) [NSU],M-CSF [NSU],M-CSF R (CD115) [NSU],Mesothelin [NSU],MIF [NSU],MMP-1 [NSU],MMP-10 [NSU],MMP-12 [NSU],MMP-2 [NSU],MMP-3 [NSU],MMP-7 [NSU],MMP-9 [NSU],NF-L [NSU],NGF beta [NSU],NRG1 beta 1 [NSU],Oncostatin M (OSM) [NSU],Osteopontin (OPN) [NSU],PCSK9 [NSU],PDGF-BB [NSU],PLGF [NSU],PTX3 (Pentraxin 3) [NSU],Resistin [NSU],SAA [NSU],SCF [NSU],ST2 (IL-33R) [NSU],TGF-beta 1 (LAP domain in precursor) [NSU],TGF-beta 1 (total) [NSU],TGF-beta 2 [NSU],TGF-beta 3 [NSU],Tie-2 [NSU],TIMP1 [NSU],Tissue Factor (TF) [NSU],TNF alpha [NSU],TNF RI [NSU],TNF RII [NSU],TNF RIII (Lymphotoxin Beta R) [NSU],TPO (Thrombopoietin) [NSU],TRAIL [NSU],TREM2 [NSU],TSLP [NSU],TWEAK [NSU],uPA [NSU],VCAM-1 [NSU],VEGF Receptor 2 (Flk-1) [NSU],VEGF-A (165) [NSU],VEGF-C [NSU],VEGF-D [NSU],VEGFR-1 [NSU],WISP-1 (CCN4) [NSU],XCL1 (Lymphotactin) [NSU]
0,0.780796,-0.402891,0.768750,-0.643275,-0.238687,-0.492554,-0.009443,-0.106441,-0.570024,-0.270803,0.043913,-0.363794,-0.169960,0.925519,0.115716,-0.292219,-0.010334,-0.317167,-0.033376,-0.134479,0.633130,0.669238,0.212007,0.701046,0.929327,0.892338,0.244735,0.740098,0.080394,0.088265,-0.567705,0.018715,-0.922841,0.984200,0.988931,0.959905,0.690908,0.944197,0.712644,0.203073,-1.102115,0.272353,-0.060136,-0.241275,-0.089792,0.100755,0.208010,0.923756,0.942710,0.801994,-0.106005,-0.011415,0.726585,-0.214893,0.683542,-0.261212,0.840677,0.871455,0.852470,-0.146625,0.945794,0.567719,-0.093503,0.177765,-0.493721,0.576935,-0.035962,-0.321261,-0.257250,-0.100388,-0.755424,-0.252667,0.251609,0.021070,-0.512643,-0.108281,0.251308,0.874977,0.790286,-0.142629,-0.032367,-0.102136,0.315371,-0.523876,0.117562,-0.083552,0.242841,0.180747,-0.120500,0.017607,0.343882,0.766426,-0.612102,-1.080372,0.668891,0.804978,0.068498,0.741621,0.697576,-0.172872,-0.020532,0.920321,-0.068696,0.182980,0.078611,0.116800,-0.104964,0.072358,-0.229357,-0.402971,-0.431158,-0.350988,0.293435,-0.015822,-0.165192,-1.407193,0.122989,0.014069,-0.402885,0.128629,0.129469,0.152757,-0.079555,-0.299054,0.172830,-0.704385,-0.101523,-0.693761,-0.120058,0.089984,0.958555,0.756085,-0.126078,0.960069,-0

In [10]:
print(nELISA_data_values["Activin A [NSU]"].mean())
print(nELISA_data_values["Activin A [NSU]"].std())

# normalize the data
scaler = preprocessing.MinMaxScaler().fit(nELISA_data_values)
nELISA_data_min_max = scaler.transform(nELISA_data_values)

nELISA_data_min_max = pd.DataFrame(
    nELISA_data_min_max, columns=nELISA_data_values.columns
)
nELISA_data_min_max.head()
print(nELISA_data_min_max["Activin A [NSU]"].mean())
print(nELISA_data_min_max["Activin A [NSU]"].std())

scaler = preprocessing.StandardScaler().fit(nELISA_data_values)
nELISA_data_standard = scaler.transform(nELISA_data_values)
nELISA_data_standard = pd.DataFrame(
    nELISA_data_standard, columns=nELISA_data_values.columns
)
nELISA_data_standard.head()
print(nELISA_data_standard["Activin A [NSU]"].mean())
print(nELISA_data_standard["Activin A [NSU]"].std())

0.01537686961452774
0.15602801101944963
0.12751334233458655
0.13825833827735404
-4.1250666807552986e-17
1.000726480299234


In [11]:
nELISA_data_values.head()

,Activin A [NSU],AITRL (GITR Ligand) [NSU],Amphiregulin [NSU],Amyloid beta [NSU],APRIL [NSU],BAFF [NSU],BCMA (TNFRSF17) [NSU],BDNF [NSU],BMP2 [NSU],BMP3 [NSU],BMP4 [NSU],BMP6 [NSU],BMP7 [NSU],BMP9 [NSU],C5/C5a [NSU],Calbindin [NSU],CCL1 [NSU],CCL11 [NSU],CCL13 [NSU],CCL15 [NSU],CCL16 [NSU],CCL17 [NSU],CCL18 [NSU],CCL19 [NSU],CCL2 [NSU],CCL20 [NSU],CCL21 [NSU],CCL22 [NSU],CCL23 [NSU],CCL24 [NSU],CCL25 [NSU],CCL27 [NSU],CCL28 [NSU],CCL3 [NSU],CCL4 [NSU],CCL5 [NSU],CCL7 [NSU],CCL8 [NSU],CD14 [NSU],CD163 [NSU],CD276 (B7-H3) [NSU],CD27L [NSU],CD30 [NSU],CD40L [NSU],CNTF [NSU],CRP [NSU],CX3CL1 [NSU],CXCL1 [NSU],CXCL10 [NSU],CXCL11 [NSU],CXCL12 (alpha) [NSU],CXCL12 (beta) [NSU],CXCL13 [NSU],CXCL14 [NSU],CXCL16 [NSU],CXCL17 [NSU],CXCL3 [NSU],CXCL4 [NSU],CXCL5 [NSU],CXCL6 [NSU],CXCL7 [NSU],CXCL9 [NSU],Cytochrome C [NSU],EGF [NSU],EGFR [NSU],EMMPRIN [NSU],FAS-L [NSU],FGF-1 [NSU],FGF-19 [NSU],FGF-2 [NSU],FGF-21 [NSU],FGF-4 [NSU],FGF-6 [NSU],FGF-7 (KGF) [NSU],FGF-9 [NSU],FGFR3 (IIIc) [NSU],FLRG (FSTL3) [NSU],Flt-3 Ligand [NSU],G-CSF [NSU],GDF-11 (BMP-11) [NSU],GDF-15 (MIC-1) [NSU],GDNF [NSU],GM-CSF [NSU],Granzyme B [NSU],Growth Hormone (Somatotropin) [NSU],HGF [NSU],HVEM [NSU],ICAM-1 [NSU],ICAM-2 [NSU],IFN alpha 2 (alpha 2b) [NSU],IFN beta [NSU],IFN gamma [NSU],IFN-epsilon [NSU],IGF-1 [NSU],IL-1 alpha [NSU],IL-1 beta [NSU],IL-1 R1 [NSU],IL-1 RA/RN [NSU],IL-10 [NSU],IL-11 [NSU],IL-12 p35 [NSU],IL-12 p40 [NSU],IL-12 p70 [NSU],IL-15 [NSU],IL-15/IL-15R alpha complex [NSU],IL-16 [NSU],IL-17A [NSU],IL-17B [NSU],IL-17C [NSU],IL-17D [NSU],IL-17E (IL-25) [NSU],IL-17F [NSU],IL-18 [NSU],IL-2 [NSU],IL-2 RA [NSU],IL-21 [NSU],IL-22 [NSU],IL-22 BP [NSU],IL-23 [NSU],IL-24 [NSU],IL-27 [NSU],IL-28A [NSU],IL-29 [NSU],IL-3 [NSU],IL-31 [NSU],IL-32 (alpha) [NSU],IL-33 [NSU],IL-35 [NSU],IL-4 [NSU],IL-5 [NSU],IL-6 [NSU],IL-6 R alpha [NSU],IL-7 [NSU],IL-8 [NSU],IL-9 [NSU],Leptin [NSU],LIF [NSU],LOX1 (OLR1) [NSU],M-CSF [NSU],M-CSF R (CD115) [NSU],Mesothelin [NSU],MIF [NSU],MMP-1 [NSU],MMP-10 [NSU],MMP-12 [NSU],MMP-2 [NSU],MMP-3 [NSU],MMP-7 [NSU],MMP-9 [NSU],NF-L [NSU],NGF beta [NSU],NRG1 beta 1 [NSU],Oncostatin M (OSM) [NSU],Osteopontin (OPN) [NSU],PCSK9 [NSU],PDGF-BB [NSU],PLGF [NSU],PTX3 (Pentraxin 3) [NSU],Resistin [NSU],SAA [NSU],SCF [NSU],ST2 (IL-33R) [NSU],TGF-beta 1 (LAP domain in precursor) [NSU],TGF-beta 1 (total) [NSU],TGF-beta 2 [NSU],TGF-beta 3 [NSU],Tie-2 [NSU],TIMP1 [NSU],Tissue Factor (TF) [NSU],TNF alpha [NSU],TNF RI [NSU],TNF RII [NSU],TNF RIII (Lymphotoxin Beta R) [NSU],TPO (Thrombopoietin) [NSU],TRAIL [NSU],TREM2 [NSU],TSLP [NSU],TWEAK [NSU],uPA [NSU],VCAM-1 [NSU],VEGF Receptor 2 (Flk-1) [NSU],VEGF-A (165) [NSU],VEGF-C [NSU],VEGF-D [NSU],VEGFR-1 [NSU],WISP-1 (CCN4) [NSU],XCL1 (Lymphotactin) [NSU]
0,0.780796,-0.402891,0.768750,-0.643275,-0.238687,-0.492554,-0.009443,-0.106441,-0.570024,-0.270803,0.043913,-0.363794,-0.169960,0.925519,0.115716,-0.292219,-0.010334,-0.317167,-0.033376,-0.134479,0.633130,0.669238,0.212007,0.701046,0.929327,0.892338,0.244735,0.740098,0.080394,0.088265,-0.567705,0.018715,-0.922841,0.984200,0.988931,0.959905,0.690908,0.944197,0.712644,0.203073,-1.102115,0.272353,-0.060136,-0.241275,-0.089792,0.100755,0.208010,0.923756,0.942710,0.801994,-0.106005,-0.011415,0.726585,-0.214893,0.683542,-0.261212,0.840677,0.871455,0.852470,-0.146625,0.945794,0.567719,-0.093503,0.177765,-0.493721,0.576935,-0.035962,-0.321261,-0.257250,-0.100388,-0.755424,-0.252667,0.251609,0.021070,-0.512643,-0.108281,0.251308,0.874977,0.790286,-0.142629,-0.032367,-0.102136,0.315371,-0.523876,0.117562,-0.083552,0.242841,0.180747,-0.120500,0.017607,0.343882,0.766426,-0.612102,-1.080372,0.668891,0.804978,0.068498,0.741621,0.697576,-0.172872,-0.020532,0.920321,-0.068696,0.182980,0.078611,0.116800,-0.104964,0.072358,-0.229357,-0.402971,-0.431158,-0.350988,0.293435,-0.015822,-0.165192,-1.407193,0.122989,0.014069,-0.402885,0.128629,0.129469,0.152757,-0.079555,-0.299054,0.172830,-0.704385,-0.101523,-0.693761,-0.120058,0.089984,0.958555,0.756085,-0.126078,0.960069,-0

In [12]:
nELISA_data_min_max.head()

,Activin A [NSU],AITRL (GITR Ligand) [NSU],Amphiregulin [NSU],Amyloid beta [NSU],APRIL [NSU],BAFF [NSU],BCMA (TNFRSF17) [NSU],BDNF [NSU],BMP2 [NSU],BMP3 [NSU],BMP4 [NSU],BMP6 [NSU],BMP7 [NSU],BMP9 [NSU],C5/C5a [NSU],Calbindin [NSU],CCL1 [NSU],CCL11 [NSU],CCL13 [NSU],CCL15 [NSU],CCL16 [NSU],CCL17 [NSU],CCL18 [NSU],CCL19 [NSU],CCL2 [NSU],CCL20 [NSU],CCL21 [NSU],CCL22 [NSU],CCL23 [NSU],CCL24 [NSU],CCL25 [NSU],CCL27 [NSU],CCL28 [NSU],CCL3 [NSU],CCL4 [NSU],CCL5 [NSU],CCL7 [NSU],CCL8 [NSU],CD14 [NSU],CD163 [NSU],CD276 (B7-H3) [NSU],CD27L [NSU],CD30 [NSU],CD40L [NSU],CNTF [NSU],CRP [NSU],CX3CL1 [NSU],CXCL1 [NSU],CXCL10 [NSU],CXCL11 [NSU],CXCL12 (alpha) [NSU],CXCL12 (beta) [NSU],CXCL13 [NSU],CXCL14 [NSU],CXCL16 [NSU],CXCL17 [NSU],CXCL3 [NSU],CXCL4 [NSU],CXCL5 [NSU],CXCL6 [NSU],CXCL7 [NSU],CXCL9 [NSU],Cytochrome C [NSU],EGF [NSU],EGFR [NSU],EMMPRIN [NSU],FAS-L [NSU],FGF-1 [NSU],FGF-19 [NSU],FGF-2 [NSU],FGF-21 [NSU],FGF-4 [NSU],FGF-6 [NSU],FGF-7 (KGF) [NSU],FGF-9 [NSU],FGFR3 (IIIc) [NSU],FLRG (FSTL3) [NSU],Flt-3 Ligand [NSU],G-CSF [NSU],GDF-11 (BMP-11) [NSU],GDF-15 (MIC-1) [NSU],GDNF [NSU],GM-CSF [NSU],Granzyme B [NSU],Growth Hormone (Somatotropin) [NSU],HGF [NSU],HVEM [NSU],ICAM-1 [NSU],ICAM-2 [NSU],IFN alpha 2 (alpha 2b) [NSU],IFN beta [NSU],IFN gamma [NSU],IFN-epsilon [NSU],IGF-1 [NSU],IL-1 alpha [NSU],IL-1 beta [NSU],IL-1 R1 [NSU],IL-1 RA/RN [NSU],IL-10 [NSU],IL-11 [NSU],IL-12 p35 [NSU],IL-12 p40 [NSU],IL-12 p70 [NSU],IL-15 [NSU],IL-15/IL-15R alpha complex [NSU],IL-16 [NSU],IL-17A [NSU],IL-17B [NSU],IL-17C [NSU],IL-17D [NSU],IL-17E (IL-25) [NSU],IL-17F [NSU],IL-18 [NSU],IL-2 [NSU],IL-2 RA [NSU],IL-21 [NSU],IL-22 [NSU],IL-22 BP [NSU],IL-23 [NSU],IL-24 [NSU],IL-27 [NSU],IL-28A [NSU],IL-29 [NSU],IL-3 [NSU],IL-31 [NSU],IL-32 (alpha) [NSU],IL-33 [NSU],IL-35 [NSU],IL-4 [NSU],IL-5 [NSU],IL-6 [NSU],IL-6 R alpha [NSU],IL-7 [NSU],IL-8 [NSU],IL-9 [NSU],Leptin [NSU],LIF [NSU],LOX1 (OLR1) [NSU],M-CSF [NSU],M-CSF R (CD115) [NSU],Mesothelin [NSU],MIF [NSU],MMP-1 [NSU],MMP-10 [NSU],MMP-12 [NSU],MMP-2 [NSU],MMP-3 [NSU],MMP-7 [NSU],MMP-9 [NSU],NF-L [NSU],NGF beta [NSU],NRG1 beta 1 [NSU],Oncostatin M (OSM) [NSU],Osteopontin (OPN) [NSU],PCSK9 [NSU],PDGF-BB [NSU],PLGF [NSU],PTX3 (Pentraxin 3) [NSU],Resistin [NSU],SAA [NSU],SCF [NSU],ST2 (IL-33R) [NSU],TGF-beta 1 (LAP domain in precursor) [NSU],TGF-beta 1 (total) [NSU],TGF-beta 2 [NSU],TGF-beta 3 [NSU],Tie-2 [NSU],TIMP1 [NSU],Tissue Factor (TF) [NSU],TNF alpha [NSU],TNF RI [NSU],TNF RII [NSU],TNF RIII (Lymphotoxin Beta R) [NSU],TPO (Thrombopoietin) [NSU],TRAIL [NSU],TREM2 [NSU],TSLP [NSU],TWEAK [NSU],uPA [NSU],VCAM-1 [NSU],VEGF Receptor 2 (Flk-1) [NSU],VEGF-A (165) [NSU],VEGF-C [NSU],VEGF-D [NSU],VEGFR-1 [NSU],WISP-1 (CCN4) [NSU],XCL1 (Lymphotactin) [NSU]
0,0.805760,0.252327,0.791864,0.201684,0.279975,0.170082,0.458191,0.366062,0.205627,0.267478,0.497775,0.033156,0.424004,0.951587,0.675322,0.480304,0.133466,0.316413,0.162975,0.338747,0.823849,0.857233,0.567917,0.891016,0.931170,0.897116,0.619747,0.791692,0.562023,0.136565,0.253001,0.147264,0.162751,0.984393,0.989102,0.961194,0.700660,0.952632,0.755347,0.719948,0.114586,0.758047,0.474386,0.146061,0.469204,0.554096,0.511781,0.925685,0.944132,0.820593,0.046379,0.212334,0.852218,0.257262,0.675567,0.375766,0.856329,0.875526,0.855539,0.466028,0.947960,0.608489,0.519554,0.570595,0.424836,0.630121,0.597200,0.414213,0.301290,0.273921,0.375920,0.342206,0.712964,0.694384,0.384164,0.321363,0.453514,0.937496,0.809189,0.361531,0.049519,0.362986,0.611441,0.257867,0.398547,0.211748,0.579714,0.403274,0.457588,0.203359,0.618125,0.772860,0.374068,0.215002,0.681757,0.812571,0.571278,0.755786,0.777220,0.250771,0.426824,0.927799,0.494399,0.402336,0.430913,0.183733,0.179884,0.555165,0.650830,0.236965,0.318459,0.210431,0.469064,0.216810,0.410315,0.301171,0.433431,0.210344,0.262167,0.527119,0.625292,0.584405,0.405155,0.231955,0.580815,0.271600,0.355548,0.310825,0.242987,0.331493,0.959757,0.785547,0.527394,0.953351,0.436309,0.743084,0.194229,0.657954,0.396899,0.435948,0.465914,0.

In [13]:
nELISA_data_standard.head()

,Activin A [NSU],AITRL (GITR Ligand) [NSU],Amphiregulin [NSU],Amyloid beta [NSU],APRIL [NSU],BAFF [NSU],BCMA (TNFRSF17) [NSU],BDNF [NSU],BMP2 [NSU],BMP3 [NSU],BMP4 [NSU],BMP6 [NSU],BMP7 [NSU],BMP9 [NSU],C5/C5a [NSU],Calbindin [NSU],CCL1 [NSU],CCL11 [NSU],CCL13 [NSU],CCL15 [NSU],CCL16 [NSU],CCL17 [NSU],CCL18 [NSU],CCL19 [NSU],CCL2 [NSU],CCL20 [NSU],CCL21 [NSU],CCL22 [NSU],CCL23 [NSU],CCL24 [NSU],CCL25 [NSU],CCL27 [NSU],CCL28 [NSU],CCL3 [NSU],CCL4 [NSU],CCL5 [NSU],CCL7 [NSU],CCL8 [NSU],CD14 [NSU],CD163 [NSU],CD276 (B7-H3) [NSU],CD27L [NSU],CD30 [NSU],CD40L [NSU],CNTF [NSU],CRP [NSU],CX3CL1 [NSU],CXCL1 [NSU],CXCL10 [NSU],CXCL11 [NSU],CXCL12 (alpha) [NSU],CXCL12 (beta) [NSU],CXCL13 [NSU],CXCL14 [NSU],CXCL16 [NSU],CXCL17 [NSU],CXCL3 [NSU],CXCL4 [NSU],CXCL5 [NSU],CXCL6 [NSU],CXCL7 [NSU],CXCL9 [NSU],Cytochrome C [NSU],EGF [NSU],EGFR [NSU],EMMPRIN [NSU],FAS-L [NSU],FGF-1 [NSU],FGF-19 [NSU],FGF-2 [NSU],FGF-21 [NSU],FGF-4 [NSU],FGF-6 [NSU],FGF-7 (KGF) [NSU],FGF-9 [NSU],FGFR3 (IIIc) [NSU],FLRG (FSTL3) [NSU],Flt-3 Ligand [NSU],G-CSF [NSU],GDF-11 (BMP-11) [NSU],GDF-15 (MIC-1) [NSU],GDNF [NSU],GM-CSF [NSU],Granzyme B [NSU],Growth Hormone (Somatotropin) [NSU],HGF [NSU],HVEM [NSU],ICAM-1 [NSU],ICAM-2 [NSU],IFN alpha 2 (alpha 2b) [NSU],IFN beta [NSU],IFN gamma [NSU],IFN-epsilon [NSU],IGF-1 [NSU],IL-1 alpha [NSU],IL-1 beta [NSU],IL-1 R1 [NSU],IL-1 RA/RN [NSU],IL-10 [NSU],IL-11 [NSU],IL-12 p35 [NSU],IL-12 p40 [NSU],IL-12 p70 [NSU],IL-15 [NSU],IL-15/IL-15R alpha complex [NSU],IL-16 [NSU],IL-17A [NSU],IL-17B [NSU],IL-17C [NSU],IL-17D [NSU],IL-17E (IL-25) [NSU],IL-17F [NSU],IL-18 [NSU],IL-2 [NSU],IL-2 RA [NSU],IL-21 [NSU],IL-22 [NSU],IL-22 BP [NSU],IL-23 [NSU],IL-24 [NSU],IL-27 [NSU],IL-28A [NSU],IL-29 [NSU],IL-3 [NSU],IL-31 [NSU],IL-32 (alpha) [NSU],IL-33 [NSU],IL-35 [NSU],IL-4 [NSU],IL-5 [NSU],IL-6 [NSU],IL-6 R alpha [NSU],IL-7 [NSU],IL-8 [NSU],IL-9 [NSU],Leptin [NSU],LIF [NSU],LOX1 (OLR1) [NSU],M-CSF [NSU],M-CSF R (CD115) [NSU],Mesothelin [NSU],MIF [NSU],MMP-1 [NSU],MMP-10 [NSU],MMP-12 [NSU],MMP-2 [NSU],MMP-3 [NSU],MMP-7 [NSU],MMP-9 [NSU],NF-L [NSU],NGF beta [NSU],NRG1 beta 1 [NSU],Oncostatin M (OSM) [NSU],Osteopontin (OPN) [NSU],PCSK9 [NSU],PDGF-BB [NSU],PLGF [NSU],PTX3 (Pentraxin 3) [NSU],Resistin [NSU],SAA [NSU],SCF [NSU],ST2 (IL-33R) [NSU],TGF-beta 1 (LAP domain in precursor) [NSU],TGF-beta 1 (total) [NSU],TGF-beta 2 [NSU],TGF-beta 3 [NSU],Tie-2 [NSU],TIMP1 [NSU],Tissue Factor (TF) [NSU],TNF alpha [NSU],TNF RI [NSU],TNF RII [NSU],TNF RIII (Lymphotoxin Beta R) [NSU],TPO (Thrombopoietin) [NSU],TRAIL [NSU],TREM2 [NSU],TSLP [NSU],TWEAK [NSU],uPA [NSU],VCAM-1 [NSU],VEGF Receptor 2 (Flk-1) [NSU],VEGF-A (165) [NSU],VEGF-C [NSU],VEGF-D [NSU],VEGFR-1 [NSU],WISP-1 (CCN4) [NSU],XCL1 (Lymphotactin) [NSU]
0,4.909214,-1.338097,3.151477,-1.585962,-1.076774,-1.793466,-0.303762,-0.335031,-1.738678,-1.360830,0.208730,-2.384336,-0.706971,3.047784,0.486010,0.056118,-0.433941,-0.852267,0.488233,-0.580848,2.134160,2.242086,1.200478,2.637008,0.469547,3.720217,0.968518,4.360853,0.480324,0.650250,-0.988302,-0.155391,-2.275325,2.658058,2.856322,1.954196,0.416020,5.248226,1.855263,1.081930,-2.917982,1.273131,-0.160971,-1.061111,-0.119956,0.171262,0.709000,4.192299,2.490506,5.067311,-1.886755,-1.569549,2.586671,-1.283677,1.707310,-0.540945,2.206722,1.851563,3.308784,0.001680,1.866454,3.072681,0.479619,0.929149,-0.839602,1.909783,0.707536,-0.802964,-0.987926,-0.090396,-0.794439,-0.580969,1.256299,1.201311,-0.841756,-0.002950,-0.105465,2.625182,4.085220,-0.236168,-1.392928,-0.278073,1.288161,-1.428026,0.694417,-1.096050,1.100885,0.402478,-0.275123,0.590197,2.322328,3.628233,-0.614804,-2.003980,2.356324,2.610452,0.005780,2.590621,4.089924,-0.608197,-0.246732,5.905284,0.299061,0.826970,0.409889,0.420542,-0.646619,0.204198,0.398590,-0.483878,-1.081099,-1.625588,1.378836,0.248769,-0.287724,-1.343899,0.794970,-0.055507,-0.856503,-0.002965,0.761974,0.527448,-0.184803,-1.440828,0.465794,-1.504582,-0.324324,-1.999497,-0.553687,1.030999,3.296927,2.417221,0.489248,1.356729,-0.4824

In [14]:
def umap_graph(df, df1):
    # UMAP parameters
    umap_df = umap.UMAP(
        n_neighbors=6,
        min_dist=0.8,
        n_components=2,
        metric="cosine",
        spread=1.1,
        init="random",
        random_state=3,
    )

    # fit and transform data
    proj_2d = umap_df.fit_transform(df)

    # plot UMAP
    fig_2d = px.scatter(
        proj_2d,
        x=0,
        y=1,
        color=df1.cell_type,
        labels={"color": "Cell Type"},
        title="UMAP projection of the nELISA data",
    ).update_layout(xaxis_title="UMAP_1", yaxis_title="UMAP_2")

    fig_2d.update_traces(marker={"size": 20})
    fig_2d.show()

    # save UMAP
    fig_2d.write_html("UMAP_projection.html")
    fig_2d.write_image("UMAP_projection.png")


umap_graph(nELISA_data_values, nELISA_data)
umap_graph(nELISA_data_min_max, nELISA_data)
umap_graph(nELISA_data_standard, nELISA_data)